## Import Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Data Import & Initial Munging
* Data was pulled from the bigquery pushshift.io dataset via SQL, with author, subreddit and count of number of posts from 2018
* Full SQL query:
    * SELECT author, subreddit, count(subreddit) FROM [fh-bigquery.reddit_posts.2018_12] WHERE author != "[deleted]" AND subreddit IS NOT NULL GROUP BY author, subreddit
* Included all months for 2018 within SQL query


In [ ]:
#import data, 3 files with all 2018 posts split into between
raw_data_1 = pd.read_csv('/Users/samholt/GA/Capstone_Data_MKII/Capstone_MKII_Data_1.csv', sep = ",")
raw_data_2 = pd.read_csv('/Users/samholt/GA/Capstone_Data_MKII/Capstone_MKII_Data_2.csv', sep = ",")
raw_data_3 = pd.read_csv('/Users/samholt/GA/Capstone_Data_MKII/Capstone_MKII_Data_3.csv', sep = ",")

In [ ]:
# merge dataframe
df_first = pd.merge(raw_data_1, raw_data_2, on=["author", "subreddit"], how= "outer" )
df= pd.merge(raw_data_3, df_first, on=["author", "subreddit"], how= "outer" )

In [ ]:
#  checking to see how the merge has gone, need to deal with null values
df.head()

In [ ]:
#dealing with null values with 0 as they mean a user hasn't posted to that subreddit in one of the three periods
df.fillna(0, inplace= True)

In [ ]:
# create one variable with overall count of posts per subreddit per user
df["posts"] = df["f0_"] + df["f0__x"] + df["f0__y"]

In [ ]:
#drop the remaining variables
df.drop(labels= ['f0_', 'f0__x', 'f0__y'], inplace= True, axis= 1)

In [ ]:
#check for null values, 
df.info()

In [ ]:
df.shape

In [ ]:
num_authors = len(df["author"].unique())
num_subreddits = len(df["subreddit"].unique())
num_posts = df["posts"].sum()

In [ ]:
labels = [num_authors, num_subreddits, num_posts]
plt.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(figsize = (8, 8))
plt.bar(x = ["Users", "Subreddits", "Posts"], height= labels, log = True)
plt.title("Unique Counts within Dataset")
plt.show()

## Data Cleaning Steps
* Converting authors to numerical key, saves memory & not interested in specific users
* Dealing with outliers in number of posts per user
* Understanding subreddit usage per user & removing those with limited information (number of subreddits posted to)
* Creating target variable
    * Boolean for whether a user posted to list of mental health related subreddits
* Merging single user subreddits and those with a small number of users in to one variable

#### Convert Authors to Numerical Key
* As we don't want to delve into individual tastes, and to save on any issues with memory converting authors to a numerical key

In [ ]:
# get list of unique authors
authors = df["author"].unique()
len(authors)

In [ ]:
# creating dictionary in order to assign numerical value per author
authors_dict = {}
counter = 0
for i in authors:
    authors_dict[i] = counter
    counter +=1

In [ ]:
# replace authors with created numerical key
df["author"] = [authors_dict[i] for i in df["author"]]

#### Checking for outliers
* Visualisation of the data shows a number of outliers in terms of number of posts per user; high posters were often bots or spam and were removed

In [ ]:
# every user has posted at least once to a specific subreddit, though seem to be a curiously high number of posts in some instances
pd.set_option('float_format', '{:f}'.format)
df.describe()

In [ ]:
# check distribution of data, we see a high number of outliers which indicate strange behaviour

fig, ax = plt.subplots(figsize=(10, 5))
plt.rcParams.update({'font.size': 16})
sns.boxplot(data=df["posts"], orient='h', fliersize=5, linewidth=3,
            saturation=0.5, ax=ax)

ax.set_title('Number of Posts per User, Pre-Outlier Removal')
ax.set_xscale("log")
plt.show()

In [ ]:
# we see there are a lot of bots included with these outliers which we will need to remove
df.sort_values(by = "posts", axis= 0, ascending= False)

In [ ]:
df_mean = df["posts"].mean()
df_std = df["posts"].std()
df_outliers = df[df["posts"] > df_mean + (df_std * 4)]

In [ ]:
df_outliers

In [ ]:
# as a large number of posters significantly higher than the mean appear to be bots or marketers, dropping them from the dataset
df = df[df["posts"] < df_mean + (df_std * 4)].copy()

In [ ]:
# after dropping outliers we see a more even distribution of data, though still a large number of outliers

fig, ax = plt.subplots(figsize=(10, 5))
plt.rcParams.update({'font.size': 16})
sns.boxplot(data=df["posts"], orient='h', fliersize=5, linewidth=3,
            saturation=0.5, ax=ax)

ax.set_title('Number of Posts per User, Post-Outlier Removal')

ax.set_xscale("log")
plt.show()

#### Understand Subreddit Usage per User
* Additionally, the majority of users only post to a small number of subreddits, any user who posts to less than 10 subreddits was dropped from the analyses


In [ ]:
# log scale
fig, ax = plt.subplots(figsize=(10, 5))
plt.rcParams.update({'font.size': 16})
df["author"].value_counts().plot(kind='hist', bins=4000, title='Number of Subreddits per User')
plt.xlabel('Author')
plt.ylabel('Frequency')
ax.set_xscale("log")
plt.show()

In [ ]:
#get dict of unique counts 
author_count = df["author"].value_counts().to_dict()

# add column for frequency of users
df["freq"] = [author_count[i] for i in df["author"]]


In [ ]:
# drop users who have posted to less than 10 subreddits, will not be useful for analysis
df = df[df["freq"] > 9].copy()

# drop frequncy column
df.drop(labels = "freq", axis= 1, inplace= True)

# del author_count to save memory
del author_count

In [ ]:
# log scale
fig, ax = plt.subplots(figsize=(10, 5))
plt.rcParams.update({'font.size': 16})
df["author"].value_counts().plot(kind='hist', bins=4000, title='Number of Subreddits per User')
plt.xlabel('Author')
plt.ylabel('Frequency')
ax.set_xscale("log")
plt.show()

#### Combine mental health related subreddits for target variable
* Create target variable
* Turn into boolean, 1 for posting to mental health subreddits

In [ ]:
#target reddits identified from r/anxiety related subreddits
string= """/r/mentalhealth/r/Anxiety/r/SuicideWatch/r/bipolar/r/bipolarreddit/r/depression/r/detachmentdisorder/r/dpdr/r/GFD/r/MentalHealth/r/mentalillness/r/stopselfharm/r/Agoraphobia/r/anxietydepression/r/HealthAnxiety/r/socialanxiety/r/Anxietyhelp/r/BPD"""
target_reddits = string.split("/r/")[1:]

In [ ]:
#get list of unique subreddits
subreddits = df["subreddit"].unique()
len(subreddits)

In [ ]:
# group together target variable
df["subreddit"] = ["df_target" if i in target_reddits else i for i in df["subreddit"]]

df[df["subreddit"] == "df_target"]

In [ ]:
#turn to boolean for prediction
df.loc[df["subreddit"] == "df_target", ["posts"]] = 1 
df[df["subreddit"] == "df_target"]

In [ ]:
#check for duplicates, boolean now so can remove
df[df["subreddit"] == "df_target"].duplicated()

In [ ]:
df.drop_duplicates(inplace= True)

In [ ]:
df[df["subreddit"] == "df_target"]

#### Merge Tiny Subreddits
* High number of subreddits have less than 10 posters, often related to personal usernames, merging these into one feature 


In [ ]:
# we see over 1 million subreddits which only have one user, merging these into "single_user_reddits"
single_df = df.groupby("subreddit").count()[df.groupby("subreddit").count()["author"] < 11]
single_df

In [ ]:
#create list of subreddits to merge
single_user_subreddits = single_df.index

In [ ]:
# renaming single user subreddits to one variable
df["subreddit"] = ["single_user_subreddits" if i in single_user_subreddits else i for i in df["subreddit"]]



In [41]:
#final file to take through for EDA
df.to_csv("/Users/samholt/GA/DSI12-lessons/projects/Capstone_Project/Capstone_MKII_DataFinal.csv'")